In [1]:
import math
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import collections as col
import re
import random
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB

In [2]:
def load_data(train='Yes', test='No', validation='No'):
	"""Loads and returns datasets as required
	   Return empty lst for if 'No'
	"""
	if train=='Yes':
		df_train = pd.read_csv('dataset/train.csv', sep=',')
	else:
		df_train = []

	if test=='Yes':
		df_test = pd.read_csv('dataset/test.csv', sep=',')
	else:
		df_test = []

	if validation=='Yes':
		df_validation = pd.read_csv('dataset/validation.csv', sep=',')
	else:
		df_validation = []
	
	print('Data loaded', len(df_train), len(df_test), len(df_validation))
	return df_train, df_test, df_validation

In [3]:
df_train, df_test, df_validation= load_data('Yes', 'No', 'Yes')

Data loaded 2697738 0 299749


In [4]:
sum(df_validation['bidprice']), 25000000, sum(df_validation['bidprice'])- 25000000

(81655340, 25000000, 56655340)

In [6]:
#Baseline CTR
b_CTR= len(df_train[df_train['click']== 1])/ len(df_train)

In [56]:
df_validation.groupby(['advertiser'])['click'].mean()
#df_validation['click'].mean()

advertiser
1458    0.000833
2259    0.000670
2261    0.000374
2821    0.000624
2997    0.004309
3358    0.000798
3386    0.000598
3427    0.000893
3476    0.000335
Name: click, dtype: float64

In [12]:
def label_encoder(df_column_nint, column_le= None): 
    """df_column_nint in form df_data[column_name]
    """
    if column_le== None:
        column_le = LabelEncoder()
        column_le.fit(df_column_nint.unique())
    if column_le!= None:
        pass
    df_column_int = column_le.transform(df_column_nint)

    return pd.DataFrame(df_column_int), column_le

In [18]:
lst_nint_features = ['le_adexchange', 'le_slotformat', 'le_slotvisibility', 'le_useragent']
#lst_int_features = ['weekday', 'hour', 'slotwidth', 'slotheight', 'advertiser']
lst_int_features = ['advertiser']

In [14]:
df_le_adexchange, le_adexc = label_encoder(df_train['adexchange'])
df_le_slotformat, le_slotf = label_encoder(df_train['slotformat'])
df_le_slotvisibility, le_slotv = label_encoder(df_train['slotvisibility'])
df_le_useragent, le_usera = label_encoder(df_train['useragent'])

df_int= df_train[lst_int_features]

In [15]:
df_int['le_adexchange'] = df_le_adexchange.values
df_int['le_slotformat'] = df_le_slotformat.values
df_int['le_slotvisibility'] = df_le_slotvisibility.values
df_int['le_useragent'] = df_le_useragent.values



/Users/jamesshields/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/jamesshields/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/jamesshields/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [16]:
df_int.columns, len(df_int)

(Index(['weekday', 'hour', 'slotwidth', 'slotheight', 'advertiser',
        'le_adexchange', 'le_slotformat', 'le_slotvisibility', 'le_useragent'],
       dtype='object'), 2697738)

In [24]:
array_y = df_train[['click']].as_matrix()
array_y_r = np.reshape(array_y, (-1, 1))

# x (int features)
lst_features= (lst_int_features+ lst_nint_features)
array_x_i = df_int[lst_int_features].as_matrix()

# Model
NB_model = GaussianNB()
NB_model.fit(array_x_i, array_y_r)

/Users/jamesshields/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [33]:
def pred_NB_model(NB_model, df_test):
    
    array_bid = df_test[['bidid']].as_matrix()

    df_le_adexchange, _ = label_encoder(df_test['adexchange'], le_adexc)
    df_le_slotformat, _ = label_encoder(df_test['slotformat'], le_slotf)
    df_le_slotvisibility, _ = label_encoder(df_test['slotvisibility'], le_slotv)
    df_le_useragent, _ = label_encoder(df_test['useragent'], le_usera)
    
    df_int= df_test[lst_int_features]
    #df_int['le_adexchange'] = df_le_adexchange.values
    #df_int['le_slotformat'] = df_le_slotformat.values
    #df_int['le_slotvisibility'] = df_le_slotvisibility.values
    #df_int['le_useragent'] = df_le_useragent.values
    
    array_x_i = df_int[lst_int_features].as_matrix()
    
    lst_predict_log_proba = []
    lst_predict = []
    lst_predict_proba = []

    lst_predict_log_proba = np.column_stack((array_bid, NB_model.predict_log_proba(array_x_i)))
    lst_predict = np.column_stack((array_bid, NB_model.predict(array_x_i)))
    lst_predict_proba =  np.column_stack((array_bid, NB_model.predict_proba(array_x_i)))

    return pd.DataFrame(lst_predict_log_proba), pd.DataFrame(lst_predict), pd.DataFrame(lst_predict_proba), array_x_i

In [47]:
df_predict_log_proba, _, df_predict_proba, features = pred_NB_model(NB_model, df_validation)

In [68]:
NB_model.predict_proba([1458])

/Users/jamesshields/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[  9.99222223e-01,   7.77776883e-04]])

In [65]:
df_validation.groupby(['advertiser'])['click'].mean()

advertiser
1458    0.000833
2259    0.000670
2261    0.000374
2821    0.000624
2997    0.004309
3358    0.000798
3386    0.000598
3427    0.000893
3476    0.000335
Name: click, dtype: float64

In [62]:
dir(NB_model)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_estimator_type',
 '_get_param_names',
 '_joint_log_likelihood',
 '_partial_fit',
 '_update_mean_variance',
 'class_count_',
 'class_prior_',
 'classes_',
 'fit',
 'get_params',
 'partial_fit',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'score',
 'set_params',
 'sigma_',
 'theta_']

In [46]:
df_predict_proba[2]/ df_predict_proba[2].mean()

0         0.988356
1          0.99415
2         0.988356
3         0.988356
4         0.988824
5         0.987821
6         0.988824
7         0.997092
8         0.987821
9          1.03154
10        0.988824
11        0.987821
12         1.03154
13        0.988824
14        0.989154
15        0.988824
16        0.988824
17         1.03154
18         1.03154
19         1.03154
20         1.00871
21        0.988824
22        0.989154
23        0.988356
24         1.03154
25        0.988824
26        0.988824
27         1.03154
28        0.989154
29        0.988356
            ...   
299719    0.987821
299720    0.988356
299721    0.988356
299722     1.03154
299723     1.03154
299724    0.988356
299725    0.997092
299726    0.989154
299727     1.00871
299728    0.989154
299729     1.03154
299730     1.03154
299731    0.989154
299732    0.987821
299733     1.03154
299734     1.03154
299735    0.987821
299736    0.987821
299737    0.988824
299738    0.988824
299739    0.988824
299740     1

In [102]:
df_predict_proba[:10]

,0,1,2
0,91c6a6b9e90c0f54d3230815a5a3e22e,0.999815,0.000184854
1,24b3621ad3b063b6c09c541781d534b3,5.70055e-05,0.999943
2,6e7bce0a9ad42f479827d37bf9618a9d,0.999805,0.000194664
3,6873feca381b8293bf418ba8408a014c,0.999776,0.000223879
4,7e6430333c552cf0b32bb21dc792d2df,0.99983,0.000169598
5,903211f027ca70f611544f42b94db094,0.9998,0.000200285
6,15274113eac79b065e186c1962c33632,0.999785,0.000214662
7,9c963e762671efbc15c4e9fb3cef9c46,0.999617,0.000382846
8,a483f16043e9f0057d542ee724cd81cb,0.999836,0.00016404
9,2da6d0891a81ac8233bba9bcffde2159,0.999827,0.000173416


In [103]:
df_predict_proba.columns = ['bid', 'prob_0', 'prob_1']

In [104]:
df_predict_proba['prob_1'].mean()

0.026881549128524345

In [105]:
df_predict_proba['prob_1n']= df_predict_proba['prob_1']/ df_predict_proba['prob_1'].mean()

In [106]:
df_predict_proba['prob_1n'].max()
len(df_predict_proba[df_predict_proba['prob_1n']> 1])

8181

In [107]:
df_predict_proba['log_prob_1']= df_predict_log_proba[2]

In [110]:
df_predict_proba[:10]

,bid,prob_0,prob_1,prob_1n,log_prob_1
0,91c6a6b9e90c0f54d3230815a5a3e22e,0.999815,0.000184854,0.00687661,-8.59594
1,24b3621ad3b063b6c09c541781d534b3,5.70055e-05,0.999943,37.1981,-5.70072e-05
2,6e7bce0a9ad42f479827d37bf9618a9d,0.999805,0.000194664,0.00724155,-8.54424
3,6873feca381b8293bf418ba8408a014c,0.999776,0.000223879,0.00832834,-8.40441
4,7e6430333c552cf0b32bb21dc792d2df,0.99983,0.000169598,0.00630907,-8.68208
5,903211f027ca70f611544f42b94db094,0.9998,0.000200285,0.00745066,-8.51577
6,15274113eac79b065e186c1962c33632,0.999785,0.000214662,0.00798548,-8.44645
7,9c963e762671efbc15c4e9fb3cef9c46,0.999617,0.000382846,0.014242,-7.86788
8,a483f16043e9f0057d542ee724cd81cb,0.999836,0.00016404,0.00610234,-8.7154
9,2da6d0891a81ac8233bba9bcffde2159,0.999827,0.000173416,0.0064511,-8.65982


In [112]:
df_predict_proba.to_csv('val_NB_pCTR_2.csv')

In [26]:
# CE loss
def ce_log_loss(df_):
    return sum(df_[['click']].values* (1- df_[['log_pCTR']].values) + \
(1- df_[['click']].values)* (df_[['log_pCTR']].values))

In [40]:
# Baseline assuming average b_CTR = 0.0007539649884458758

df_validation_pCTR_baseline= df_validation
df_validation_pCTR_baseline['pCTR']= b_CTR
df_validation_pCTR_baseline['log_pCTR'] = np.log(df_validation_pCTR_baseline['pCTR'].values)
print(ce_log_loss(df_validation_pCTR_baseline))
print(df_validation_pCTR_baseline['pCTR'].mean())

[-2151768.70189764]
0.000753964988441278


In [29]:
LR_model1 = pd.read_csv('LR_pCTR_validation.csv')
NB_model1 = pd.read_csv('val_NB_pCTR_1.csv')
NB_model2 = pd.read_csv('val_NB_pCTR_2.csv')

In [36]:
df_validation_pCTR_NB= df_validation
df_validation_pCTR_NB['bidid_2'] = NB_model1['bid'].values
df_validation_pCTR_NB['pCTR'] = NB_model1['prob_1'].values
df_validation_pCTR_NB['log_pCTR'] = NB_model1['log_prob_1'].values
print(ce_log_loss(df_validation_pCTR_NB))
print(df_validation_pCTR_NB['pCTR'].mean())

[-2392425.90439859]
0.026974865223408976


In [34]:
df_validation_pCTR_NB_2= df_validation
df_validation_pCTR_NB_2['bidid_2'] = NB_model2['bid'].values
df_validation_pCTR_NB_2['pCTR'] = NB_model2['prob_1'].values
df_validation_pCTR_NB_2['log_pCTR'] = NB_model2['log_prob_1'].values
print(ce_log_loss(df_validation_pCTR_NB_2))
print(df_validation_pCTR_NB_2['pCTR'].mean())

[-2440614.66430027]
0.026881549128524345


In [35]:
df_validation_pCTR_LR= df_validation
df_validation_pCTR_LR['pCTR']= LR_model1['1'].values
df_validation_pCTR_LR['log_pCTR']= np.log(df_validation_pCTR_LR['pCTR'].values)
print(ce_log_loss(df_validation_pCTR_LR))
print(df_validation_pCTR_LR['pCTR'].mean())

[-2254155.39045959]
0.0007536088052850036
